In [24]:
import cv2
import numpy as np

class ImageAligner:
    def __init__(self, image_path=None, image_data=None, rect=None):
        if image_path:
            self.image = cv2.imread(image_path)
        elif image_data:
            self.image = image_data
        elif rect:
            self.image = np.zeros((rect[3], rect[2], 3), np.uint8)
        else:
            raise ValueError("Either image_path, image_data, or rect must be provided")

    def align_by_content(self):
        gray = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)

        sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
        sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)

        magnitude, angle = cv2.cartToPolar(sobel_x, sobel_y, angleInDegrees=True)

        hist, bins = np.histogram(angle.flatten(), bins=180, range=(0, 180))
        dominant_angle = bins[np.argmax(hist)]

        (h, w) = self.image.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, dominant_angle, 1.0)
        aligned_image = cv2.warpAffine(self.image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

        return aligned_image


aligner = ImageAligner(image_path='Project1.jpg')
aligned_image = aligner.align_by_content()
cv2.imshow('Aligned Image', aligned_image)
cv2.waitKey(0)
cv2.destroyAllWindows()